### This python notebook aims to collect data on teams and players from the web

#### Links

In [1]:
# S3 bucket includes
val_esports_fandom_url = 'https://valorant-esports.fandom.com/wiki/VALORANT_Esports_Wiki'
val_fandom_url = 'https://valorant.fandom.com/wiki/VALORANT'

tracker.gg
tracker_gg_url = 

#### Packages

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import re

#### Liquipedia

In [3]:
# Liquidedia valorant
liquipedia_url_base = 'https://liquipedia.net'
val_liquipedia_url = 'https://liquipedia.net/valorant/'

In [4]:
from bs4 import BeautifulSoup
import requests
from liquipediaFunctions import get_player_names
from liquipediaFunctions import get_player_urls
from liquipediaFunctions import player_scrape

liquipedia_val_team_portal_url = val_liquipedia_url + 'Portal:Teams#Notable_Active_Teams'
liquipedia_team_portal_page = requests.get(liquipedia_val_team_portal_url)
liquipedia_soup = BeautifulSoup(liquipedia_team_portal_page.text, 'html.parser')

liquipedia_teams = liquipedia_soup.findAll('span', class_ = 'team-template-text')
liquipedia_teams_url_list = [team.find('a')['href'] for team in liquipedia_teams]

In [ ]:
liquipedia_teams

In [ ]:
liquipedia_teams_url_list

In [7]:
# liquipedia_players = liquipedia_soup.findAll('td') # style_ = 'font-weight:bold')
liquipedia_player_text = liquipedia_soup.find_all('span', attrs={'style': 'white-space:pre'})


liquipedia_player_urls = get_player_urls(liquipedia_player_text)
liquipedia_player_names = get_player_names(liquipedia_player_text)

In [ ]:
liquipedia_player_urls

In [ ]:
liquipedia_player_names

In [ ]:
player_url = '/valorant/Asuna'
headers = player_scrape(player_url, '/Matches')
print(headers)
cleaned = headers[:5] + headers[6:]
print(cleaned)

Players, teams, agents*, roles*, 

#### Valorant.gg

In [ ]:
# Specifically, VCT International
# Insights for valorant team roles = 'https://www.thespike.gg/valorant/news/competitive-team-roles-in-valorant/142'

In [2]:
vlr_gg_url = 'https://www.vlr.gg/stats'
filter_hyperlink = "/?event_group_id=all&event_id=%s&series_id=all&region=all&min_rounds=0&min_rating=0&agent=all&map_id=all&timespan=all"

In [3]:
page = requests.get(vlr_gg_url)
soup = BeautifulSoup(page.text, 'html.parser')
tags = soup.find_all('option')

In [4]:
game_changers = [(tag['value'], tag.text) for tag in tags if ('Game Changers' in tag.text and len(tag['value']) > 2)]
game_changers = game_changers[0:-2] ## Last two Results had issues with the link

In [ ]:
game_changers # For testing

In [30]:
# To be transferred in vlrgg API afterwards 
for event_id, event_name in game_changers:
    event_filter_hyperlink = filter_hyperlink % event_id
    event_page = requests.get(vlr_gg_url + event_filter_hyperlink)
    event_soup = BeautifulSoup(event_page.text, 'html.parser')
    headers = event_soup.find_all('th')

In [10]:
## This chunk is for testing purposes

link = 'https://www.vlr.gg/stats/?event_group_id=all&event_id=2153&series_id=all&region=all&min_rounds=0&min_rating=0&agent=all&map_id=all&timespan=all'
page = requests.get(link)
soup = BeautifulSoup(page.text, 'html.parser')
entries = soup.find_all('tr')

In [11]:
table_body = soup.find('table', class_='wf-table mod-stats mod-scroll').find('tbody')
table_rows = table_body.find_all('tr')

# Step 3: Extract data from each <tr> and <td> to form a list of lists
data = []
for row in table_rows:
    row_data = []
    cols = row.find_all('td')
    # Extract text or other necessary data from each column
    for col in cols:
        col_class = col.get('class', [])
        if col_class == ['mod-player', 'mod-a']:
            player_name = col.find('a').get('href', 'NaN').split('/')[-1]
            country =  col.find('i').get('class', 'NaN')[-1].split('-')[-1]
            row_data.append(player_name)
            row_data.append(country)
        elif col_class == ['mod-agents']:
            agents = []
            images = col.find_all('img')
            for image in images:
                src = image['src']
                agent_name_match = re.search(r'agents/(\w+)\.png', src)
                if agent_name_match:
                    agent_name = agent_name_match.group(1)
                    agents.append(agent_name)
                else:
                    agents.append('NaN')
            row_data.append(agents)
        else:
            text = col.text
            if text == '':
                value = 'NaN'
            else:
                value = text.strip()
            row_data.append(value)
    while len(row_data) < 22:
        row_data.append('NaN')  # Fill missing columns with 'NaN'
    data.append(row_data)

# Step 4: Create a DataFrame from the extracted data
columns = ['Player', 'Country', 'Agents', 'RND', 'RATING', 'ACS',\
        'K:D', 'KAST', 'ADR', 'KPR', 'APR', 'FKPR', 'FDPR',\
        'HS%', 'CL%', 'CL', 'KMAX', 'K', 'D', 'A', 'FK', 'FD']  # Update column names as needed
df = pd.DataFrame(data, columns=columns)

# Display the DataFrame
df

,Player,Country,Agents,RND,RATING,ACS,K:D,KAST,ADR,KPR,...,FDPR,HS%,CL%,CL,KMAX,K,D,A,FK,FD
0,theia,us,"[astra, omen]",27,1.76,271.0,4.29,89%,159.4,1.11,...,0.04,36%,NaN,,15,30,7,21,2,1
1,florescent,ca,"[raze, jett, neon]",254,1.56,324.1,1.92,82%,203.9,1.13,...,0.09,25%,21%,4/19,28,286,149,65,57,23
2,unstable,kr,"[sova, fade]",39,1.47,299.5,1.59,77%,190.3,1.10,...,0.08,29%,NaN,0/5,22,43,27,18,4,3
3,rayne,pr,"[neon, raze]",49,1.42,304.0,1.50,73%,169.3,1.04,...,0.16,27%,22%,2/9,27,51,34,5,9,8
4,dodonut,th,"[raze, jett, sage]",246,1.38,282.7,1.59,82%,179.6,0.98,...,0.11,22%,25%,3/12,30,240,151,77,50,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,inthemood,un,"[sova, raze, kayo]",68,0.46,130.8,0.44,43%,94.2,0.40,...,0.15,25%,NaN,0/5,9,27,62,12,4,10
232,yun,un,"[cypher, neon]",73,0.43,128.0,0.50,53%,87.7,0.45,...,0.19,34%,NaN,0/6,15,33,66,12,3,14
233,light,us,"[viper, omen]",70,0.43,112.5,0.42,40%,72.2,0.39,...,0.13,15%,NaN,0/7,12,27,65,15,2,9
234,shinx,un,"[sova, gekko]",92,0.42,99.8,0.35,48%,70.2,0.30,...,0.14,21%,10%,1/10,9,28,79,13,4,13
